<a href="https://colab.research.google.com/github/widyadaridhyp/Lomba/blob/main/prescriptive_analytics_part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import Libraries**

In [ ]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%pip install lazypredict

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from nltk.tokenize import word_tokenize

In [ ]:
def my_function():
    %pip install nltk
    import nltk
    # Rest of your code that uses nltk

# Other parts of your code

In [ ]:
import nltk

AttributeError: ignored

In [ ]:

import re
import spacy
import plotly.express as px
import plotly.graph_objects as go
import io
from plotly.offline import plot, iplot, init_notebook_mode
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from scipy.stats import norm, skew # for some statistics
import warnings # to ignore warning
from sklearn.preprocessing import RobustScaler, PowerTransformer, LabelEncoder
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV

import lazypredict
from lazypredict.Supervised import LazyRegressor
import optuna
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LassoCV, RidgeCV

from sklearn.metrics import mean_absolute_error
import sklearn.metrics as metrics
from sklearn.metrics import r2_score, mean_squared_error

from sklearn.ensemble import StackingRegressor, RandomForestRegressor, GradientBoostingRegressor
import xgboost as xgb
import lightgbm as lgb
import joblib

# **Read Data**

In [ ]:
target_df = pd.read_csv("train_label.csv")
train_df = pd.read_csv("train_features.csv")

In [ ]:
# Add a key column to each dataset
train_df['key'] = range(len(train_df))
target_df['key'] = range(len(target_df))

# Merge the datasets using the key column
master_df = pd.merge(train_df, target_df, on='key')

# Remove the key column
master_df = master_df.drop('key', axis=1)

master_df

,facilities,rating,location,Price
0,RestaurantBARSwimmingPools,7.8 Very GoodFrom 10 reviews,Stokol,"13,500avg/night"
1,intrnetRestaurantgym,5.6 GoodFrom 4 reviews,Machlessvile,"13,000avg/night"
2,restaurantgympoolBar,7.2 Very GoodFrom 38 reviews,Wanderland,"19,000avg/night"
3,BARRestaurant,7.3 Very GoodFrom 6 reviews,Uberlandia,"6,000avg/night"
4,InternetRestaurant,7.2 Very GoodFrom 30 reviews,Stokol,"20,000avg/night"
...,...,...,...,...
3061,barInternet,NaN,Andeman,"31,625avg/night"
3062,restaurantBarInternet,8.1 ExcellentFrom 4 reviews,Uberlandia,"30,500avg/night"
3063,Barrestaurantswimmingpools,6.7 Very GoodFrom 10 reviews,Willsmian,"14,000avg/night"
3064,Restaurant,NaN,Hallerson,"8,500avg/night"


In [ ]:
# Size Dataset
print('Size dataset:', master_df.shape)

Size dataset: (3066, 4)


In [ ]:
# Data Overview
master_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3066 entries, 0 to 3065
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   facilities  2765 non-null   object
 1   rating      2429 non-null   object
 2   location    3066 non-null   object
 3   Price       3066 non-null   object
dtypes: object(4)
memory usage: 119.8+ KB


# **Data Preprocessing**

# Feature Engineering

In [ ]:
# Change type obj to str
master_df = master_df.astype({"facilities":"str","rating":"str","Price":"str"})

Ubah tipe data untuk features 'facilities', 'rating', dan 'Price' dari object ke string supaya dapat digunakan modul nltk

###**Cleaning Feature 'rating'**

In [ ]:
def my_function()
# Download required resources from NLTK
nltk.download('punkt')

# Define a function to extract rating, review, and total reviews
def extract_info(text):
    tokens = nltk.word_tokenize(str(text))  # Convert to string
    rating = tokens[0]
    review = tokens[1:-2]
    total_reviews = tokens[-2] if len(tokens) >= 2 else None
    return rating, review, total_reviews

# Apply the function to the 'rating' column
master_df[['rating', 'review', 'total reviews']] = master_df['rating'].apply(extract_info).apply(pd.Series)

master_df

SyntaxError: ignored

Pisahkan data pada feature 'rating' menjadi 3 features baru yaitu 'rating' yang berisi rating pengunjung terhadap Kyozo hotel, 'review' yaitu keseluruhan review yang diberikan pengunjung, dan 'total reviews' yaitu banyaknya pengunjung yang melakukan review.

In [ ]:
# Replace NaN, empty list, and None with NaN
master_df['rating'] = master_df['rating'].replace('nan', np.nan)
master_df['review'] = master_df['review'].apply(lambda x: np.nan if len(x) == 0 else x)
master_df['total reviews'] = master_df['total reviews'].apply(lambda x: np.nan if x is None else x)

master_df

KeyError: ignored

###**Cleaning Feature 'facilities'**

In [ ]:
# Define the dictionary of facilities
facilities_dict = {
    'restaurant': ['restaurant', 'food'],
    'bar': ['bar'],
    'swimming pool': ['pool', 'swimming', 'pools'],
    'gym': ['gym', 'fitness'],
    'internet': ['internet', 'intrnet']
}

# Function to separate the text into facilities based on the dictionary
def separate_facilities(text):
    separated_text = []
    for facility, keywords in facilities_dict.items():
        if any(keyword in text.lower() for keyword in keywords):
            separated_text.append(facility)
    return ', '.join(separated_text)

# Apply the separation function to the 'facilities' column
master_df['facilities'] = master_df['facilities'].apply(separate_facilities)

master_df

,facilities,rating,location,Price
0,"restaurant, bar, swimming pool",7.8 Very GoodFrom 10 reviews,Stokol,"13,500avg/night"
1,"restaurant, gym, internet",5.6 GoodFrom 4 reviews,Machlessvile,"13,000avg/night"
2,"restaurant, bar, swimming pool, gym",7.2 Very GoodFrom 38 reviews,Wanderland,"19,000avg/night"
3,"restaurant, bar",7.3 Very GoodFrom 6 reviews,Uberlandia,"6,000avg/night"
4,"restaurant, internet",7.2 Very GoodFrom 30 reviews,Stokol,"20,000avg/night"
...,...,...,...,...
3061,"bar, internet",NaN,Andeman,"31,625avg/night"
3062,"restaurant, bar, internet",8.1 ExcellentFrom 4 reviews,Uberlandia,"30,500avg/night"
3063,"restaurant, bar, swimming pool",6.7 Very GoodFrom 10 reviews,Willsmian,"14,000avg/night"
3064,restaurant,NaN,Hallerson,"8,500avg/night"


Bersihkan data ada feature 'facilities' dengan memisahkan fasilitas-fasilitas yang ada pada Kyozo hotel menjadi data yang terurut.

In [ ]:
# Identify distinct values in the 'facilities' feature
distinct_values = master_df['facilities'].str.split(', ').explode().unique()

print(distinct_values)

In [ ]:
# Replace empty strings with NaN
master_df['facilities'] = master_df['facilities'].replace('', np.nan)
master_df

In [ ]:
# Feature engineering: 'have bar'
master_df['have bar'] = master_df['facilities'].apply(lambda x: 2 if pd.isna(x) else (1 if 'bar' in x else 0))

# Feature engineering: 'have restaurant'
master_df['have restaurant'] = master_df['facilities'].apply(lambda x: 2 if pd.isna(x) else (1 if 'restaurant' in x else 0))

# Feature engineering: 'have swimming pools'
master_df['have swimming pool'] = master_df['facilities'].apply(lambda x: 2 if pd.isna(x) else (1 if 'swimming pool' in x else 0))

# Feature engineering: 'have gym'
master_df['have gym'] = master_df['facilities'].apply(lambda x: 2 if pd.isna(x) else (1 if 'gym' in x else 0))

# Feature engineering: 'have internet'
master_df['have internet'] = master_df['facilities'].apply(lambda x: 2 if pd.isna(x) else (1 if 'internet' in x else 0))

master_df

###**Cleaning Feature 'Price'**

In [ ]:
# Remove non-numeric characters from the 'Price' column
master_df['Price'] = master_df['Price'].str.replace(',', '').str.extract('(\d+)')

# Convert the 'Price' column to integer type
master_df['Price'] = master_df['Price'].astype(int)

master_df

# Handle Missing Values

In [ ]:
master_df.info()

In [ ]:
# Missing Value
print('The number of missing value on dataset:')
master_df.isna().sum()

In [ ]:
# Check percentage of missing values
master_df.isna().sum()/len(master_df)*100

In [ ]:
# Plot missing values
sns.heatmap(master_df.isna(),yticklabels=False,cbar=False,cmap='viridis')

Karena terdapat banyak missing value pada features 'facilities' dan 'rating', maka kita perlu menemukan hubungan antara missing value dengan variabel target 'price'.

# Mengganti Data Missing Values

In [ ]:
# Nan: Diasumsikan lupa / tidak tahu fasilitas apa saja yang ada
master_df.facilities.fillna('tidak tahu', inplace=True)

In [ ]:
# Boxplot the 'rating' column data
# Replace 'From' with NaN
master_df['rating'] = master_df['rating'].replace('From', pd.NA)

# Drop rows with NaN values in the 'rating' column
df_rating = master_df['rating'].dropna()

# Convert the 'rating' column to float
df_rating = df_rating.astype(float)

sns.boxplot(df_rating)
plt.show()

In [ ]:
# Replace missing values in the 'rating' feature with the mean of the available data
master_df['rating'] = master_df['rating'].replace(np.nan, df_rating.mean())

In [ ]:
# Boxplot the 'total_reviews' column data
# Drop rows with NaN values in the 'rating' column
df_totalreviews = master_df['total reviews'].dropna()

# Convert the 'total_reviews' column to int
df_totalreviews = df_totalreviews.astype(int)

sns.boxplot(df_totalreviews)
plt.show()

In [ ]:
# Calculate the mean of the 'total reviews' column
mean_total_reviews = df_totalreviews.mean()

# Replace the missing values with the mean and apply ceiling
master_df['total reviews'] = master_df['total reviews'].replace(np.nan, np.ceil(mean_total_reviews))

In [ ]:
# Nan: Diasumsikan masuk kategori 'Very Good' beradasrkan mean yang telah diperoleh
master_df.review.fillna("['Very', 'GoodFrom']", inplace=True)

In [ ]:
# Plot missing values
sns.heatmap(master_df.isna(),yticklabels=False,cbar=False,cmap='viridis')

Berdasarkan heatmap diatas data sudah bersih dari missing values karena missing values sudah berhasil diubah berdasarkan asumsi dan modus data

# Labeling Input Data

In [ ]:
# Convert all elements in 'review' column to strings
master_df['review'] = master_df['review'].astype(str)

# Identify distinct values in the 'review' feature
distinct_values_review = master_df['review'].unique()

print(distinct_values_review)

In [ ]:
# Identify distinct values in the 'location' feature
distinct_values_loc = master_df['location'].unique()

print(distinct_values_loc)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

# Fit and transform the 'review' column using LabelEncoder
le.fit(master_df['review'].drop_duplicates())
master_df['review'] = le.transform(master_df['review'])

# Fit and transform the 'location' column using LabelEncoder
le.fit(master_df['location'].drop_duplicates())
master_df['location'] = le.transform(master_df['location'])

In [ ]:
master_df

Labeling feature 'review'

*   0 - Excellent
*   1 - Fair
*   2 - God
*   3 - Very Good

Labeling feature 'location'

*   0 - Andeman
*   1 - Hallerson
*   2 - Machlessvile
*   3 - Stokol
*   4 - Uberlandia
*   5 - Ubisville
*   6 - Wanderland
*   7 - Willsmian

In [ ]:
master_df.info()

In [ ]:
# Change type data
master_df = master_df.astype({"rating":"float","total reviews":"int"})
# Drop irelevant feature
master_df.drop(['facilities'], axis = 1, inplace = True)

# Lazy Predictions

Lazy Predict merupakan library Python yang menyediakan cara sederhana dan efisien untuk membuat prediksi. Lazy Predict adalah tools yang dapat memperlihatkan akurasi dari berbagai model untuk mempermudah proyek pemodelan prediktif.

In [ ]:
X = master_df[['rating', 'location', 'review', 'total reviews',
       'have bar', 'have restaurant', 'have swimming pool', 'have gym',
       'have internet']]
y = master_df['Price']
X = X.astype(np.float32)
offset = int(X.shape[0] * 0.9)
X_train, y_train = X[:offset], y[:offset]
X_test, y_test = X[offset:], y[offset:]
reg = LazyRegressor(verbose=0,ignore_warnings=False, custom_metric=None )
models,predictions = reg.fit(X_train, X_test, y_train, y_test)
models

# **Exploratory Data Analysis**



In [ ]:
# Change type data
eda_df = master_df.astype({"rating":"float","location":"object","review":"object","total reviews":"int","have bar":"object",
                              "have restaurant":"object","have swimming pool":"object","have gym":"object","have internet":"object"})

In [ ]:
# Unique Columns
eda_df.select_dtypes(include='object').nunique()

Features di atas memiliki tipe data object dengan yang memiliki kategori dengan jumlah masing-masing dapat dilihat diatas.

In [ ]:
# Check duplicate values
eda_df.duplicated().sum()

Terdapat 436 data yang duplicate

In [ ]:
# Uji korelasi atribut
f, ax = plt.subplots(figsize=(10, 10))
corr = master_df.corr()
hm = sns.heatmap(round(corr,2), annot=True, ax=ax, cmap="coolwarm", fmt='.2f', linewidths=.05)

Berdasarkan korelasi matriks, bisa didapatkan bahwa :


1.   Dengan nilai koefisien korelasi 0.6, terdapat korelasi yang kuat antara rekomendasi dokter pada vaksin H1N1 dan rekomendasi dokter pada vaksin flu seasonal.
2.   Dengan nilai koefisien korelasi 0.56, terdapat korelasi sedang antara opini responden terhadap resiko vaksin H1N1 dan opini responden terhadap resiko vaksin flu seasonal.
3.   Dengan nilai koefisien korelasi 0.48, terdapat korelasi sedang antara opini responden terhadap keefektifan vaksin H1N1 dan opini responden terhadap keefektifan vaksin flu seasonal.
4.   Terdapat korelasi antara sudahkah responden mendapatkan vaksinasi H1N1 dengan rekomendasi dokter pada vaksin H1N1, opini responden terhadap keefektifan vaksin H1N1, opini responden terhadap resiko flu H1N1, dan sudahkan responden mendapatkan vaksinasi seasonal flu.
5.   Terdapat korelasi antara sudahkah responden mendapatkan vaksinasi flu seasonal dengan rekomendasi dokter pada vaksin seasonal, opini responden terhadap keefektifan vaksin flu seasonal, opini responden terhadap resiko flu seasonal, kelompok usia, dan sudahkan responden mendapatkan vaksinasi H1N1.







In [ ]:
# Membuat grafik rata-rata harga hotel dengan total reviews dan review
fig = px.scatter(eda_df, x='total reviews', y='Price', color='review', size='Price',
                 title="Hotel Price increase with Total Reviews among Review",
                 color_discrete_sequence=['#B14B51','#B7A294'],height=600)
fig.update_layout(legend=dict(title='',orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
                  font_color="#303030", xaxis=dict(title='Total Reviews',showgrid=False),
                  yaxis=dict(title='Hotel Price, $',showgrid=False, zerolinecolor='#E5E5EA',
                             showline=True, linecolor='#E5E5EA', linewidth=2))
fig.show()

# Categorical Data

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# get list of categorical variables
categoric = ['location', 'review', 'have bar', 'have restaurant', 'have swimming pool', 'have gym', 'have internet']

In [ ]:
# Menunjukan statistika deskriptif dari variabel kategorik di dataset
# Filter the DataFrame to include only the categorical variables
categorical_df = eda_df[categoric]

# Use describe() on the categorical DataFrame
categorical_df.describe()

In [ ]:
# Menunjukan rata-rata harga hotel dari variabel kategorik
for i in categoric:
    obs=eda_df[i].value_counts()
    avg_price=eda_df.groupby(i)["Price"].mean()
    display(pd.DataFrame({"Number of Visitors":obs, "Average Price Hotel":avg_price.map('${:,.2f}'.format)})\
            .sort_values('Number of Visitors', ascending=False)\
            .style.set_caption("Variable: {}".format(i)))

In [ ]:
# create figure with subplots
fig, axs = plt.subplots(nrows = 2, ncols = 4, figsize = (20,10))
axs = axs.flatten()

# create Pie Chart for each categorical variable
for i, var in enumerate(categoric):
  counts = eda_df[var].value_counts()
  labels = counts.index.tolist()
  values = counts.values.tolist()
  axs[i].pie(values, labels=labels, autopct='%1.1f%%')
  axs[i].set_title(var)

# adjust spacing between subplots
fig.tight_layout()

# show plot
plt.show()

In [ ]:
# Visualize Categorical Data
# Initialize subplot
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(20, 10))
x_position = 0
y_position = 0

for i in categoric:
    if i == 'review':
        continue  # Skip the 'review' variable
    if y_position == 3:
        y_position = 0
        x_position += 1
    df_plot = eda_df.groupby(['review', i]).size().reset_index().pivot(columns='review', index=i, values=0)
    ax = df_plot.plot(kind='bar', stacked=True, ax=axes[x_position, y_position])
    ax.set_title(i)
    plt.xticks(rotation=90)
    ax.legend(loc='upper right')
    y_position += 1

plt.show()

In [ ]:
# create figure with subplots
fig, axs = plt.subplots(nrows = 2, ncols = 3, figsize = (20,10))
axs = axs.flatten()

# create histplot for each categorical variable
cat = ['location', 'have bar', 'have restaurant', 'have swimming pool', 'have gym', 'have internet']
for i, var in enumerate(cat):
  sns.histplot(x = var, hue = 'review', data = eda_df, ax=axs[i], multiple = 'fill', kde = False)
  axs[i].set_xticklabels(eda_df[var].unique(), rotation = 90)
  axs[i].set_xlabel(var)

# adjust spacing between subplots
fig.tight_layout()

# show plot
plt.show()

In [ ]:
# Membuat grafik distribusi harga hotel dengan review
fig=px.histogram(eda_df, x='Price', color='review', opacity=0.7, barmode='overlay',
                 histnorm='probability density', marginal='box',
                 title="Distribution of Hotel Price by Review Category",
                 color_discrete_sequence=['#B14B51','#B7A294'])
fig.update_layout(font_color="#303030", xaxis_title='Hotel Price, $',
                  yaxis=dict(title='Probability Density', gridcolor='#EAEAEA', zerolinecolor='#EAEAEA'),
                  legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1, title=""))
fig.update_xaxes(showgrid=False, zerolinecolor='#EAEAEA')
fig.show()

In [ ]:
# Membuat grafik harga hotel dengan lokasi dan review
plot_df = eda_df.groupby(['location','review'])['Price'].mean()
plot_df = plot_df.rename('Price').reset_index()
fig = px.bar(plot_df, x='location', y='Price', color='review', height=800,
             title="Average Hotel Price by Location and Review",
             #color_discrete_map={'0': '#BFC5DA','1': '#5D6A92', '2': '#808080', '3': '#800080'},
             facet_row='review', text='Price', opacity=0.85, barmode='group')
fig.update_traces(texttemplate='$%{text:,.0f}', textposition='outside',
                  marker_line=dict(width=1, color='#303030'))
fig.layout.yaxis2.update(matches=None)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.update_xaxes(title="location", row=1)
fig.update_yaxes(title="Hotel Price, $",  gridcolor='#E3E3E3', zeroline=True, zerolinewidth=2,
                 showgrid=False, zerolinecolor='#E5E5EA',
                 showline=True, linecolor='#E5E5EA', linewidth=2)
fig.update_layout(font_color="#303030", paper_bgcolor="white", plot_bgcolor="white",
                  bargroupgap=0.05, bargap=0.2,
                  legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1, title=""))
fig.show()

In [ ]:
num_loc = master_df['location'].value_counts().sort_values(ascending=False)[0:20]
num_loc.name = 'Count'
num_loc.index.name = 'Location'

In [ ]:
fig = plt.figure(figsize=(10,5))
sns.barplot(x=num_loc.index,y=num_loc)
plt.tight_layout()

In [ ]:
# draw a boxen of a categorical variable along with
# how it effects the target variable which is in this case 'price'
def draw_cate_vs_target(df, feature):
    plt.grid(True)
    ax = sns.boxenplot(data=df, x=feature, y='Price')
    ax.set_title(f'{feature} vs price', fontdict={'fontsize': 18})

# draw the countplot of a categorical variable
def draw_cate_countplot(df, feature):
    plt.grid(True)
    ax = sns.countplot(x=df[feature])
    ax.set_title(f'{feature}\'s count', fontdict={'fontsize': 18})

In [ ]:
# draw a boxen plot of bar vs price
plt.figure(figsize=(20, 6))
sns.set_palette('Set2')
plt.subplot(1, 2, 1)
draw_cate_vs_target(master_df, categoric[2])

plt.subplot(1, 2, 2)
draw_cate_countplot(master_df, categoric[2])
plt.show()

In [ ]:
# draw a boxen plot of restaurant vs price
plt.figure(figsize=(20, 6))
sns.set_palette('Set2')
plt.subplot(1, 2, 1)
draw_cate_vs_target(master_df, categoric[3])

plt.subplot(1, 2, 2)
draw_cate_countplot(master_df, categoric[3])
plt.show()

In [ ]:
# draw a boxen plot of swimming pool vs price
plt.figure(figsize=(20, 6))
sns.set_palette('Set2')
plt.subplot(1, 2, 1)
draw_cate_vs_target(master_df, categoric[4])

plt.subplot(1, 2, 2)
draw_cate_countplot(master_df, categoric[4])
plt.show()

In [ ]:
# draw a boxen plot of gym vs price
plt.figure(figsize=(20, 6))
sns.set_palette('Set2')
plt.subplot(1, 2, 1)
draw_cate_vs_target(master_df, categoric[5])

plt.subplot(1, 2, 2)
draw_cate_countplot(master_df, categoric[5])
plt.show()

In [ ]:
# draw a boxen plot of internet vs price
plt.figure(figsize=(20, 6))
sns.set_palette('Set2')
plt.subplot(1, 2, 1)
draw_cate_vs_target(master_df, categoric[6])

plt.subplot(1, 2, 2)
draw_cate_countplot(master_df, categoric[6])
plt.show()

# Numerical Data

In [ ]:
# Menunjukan statistika deskriptif dari variabel numerik di dataset
master_df.describe().T

In [ ]:
# get list of numerical variables
numeric = ['rating', 'total reviews', 'Price']

In [ ]:
fig, axs = plt.subplots(nrows = 3, ncols = 1, figsize = (10,10))
axs = axs.flatten()

# create boxplot for each numerical variable
for i, var in enumerate(numeric):
  sns.boxplot(x = var, data = master_df, ax=axs[i])

# adjust spacing between subplots
fig.tight_layout()

# show plot
plt.show()

In [ ]:
fig, axs = plt.subplots(nrows = 3, ncols = 1, figsize = (10,10))
axs = axs.flatten()

# create boxplot for each numerical variable
for i, var in enumerate(numeric):
  sns.violinplot(x = var, data = master_df, ax=axs[i])

# adjust spacing between subplots
fig.tight_layout()

# show plot
plt.show()

In [ ]:
fig, axs = plt.subplots(nrows = 3, ncols = 1, figsize = (10,10))
axs = axs.flatten()

# create boxplot for each numerical variable
for i, var in enumerate(numeric):
  sns.histplot(x = var, data = master_df, ax=axs[i])

# adjust spacing between subplots
fig.tight_layout()

# show plot
plt.show()

In [ ]:
def bivariate_analysis(x):
  plt.figure(figsize=(8,4))
  ax = sns.regplot(x=x, y='Price',data=master_df)
  ax.set_title("Price vs "+x, fontsize=15)
  ax.set_xlabel(x, fontsize=10)
  ax.set_ylabel('Price', fontsize=10)
  plt.locator_params(axis='x', nbins=10)

In [ ]:
for x in numeric:
  if x == 'Price':
    continue  # Skip the 'review' variable
  bivariate_analysis(x)
plt.tight_layout()

In [ ]:
sns.pairplot(master_df[numeric],diag_kind='kde')

In [ ]:
eda_df.groupby('rating')['Price'].median().plot()
plt.xlabel('Rating')
plt.ylabel('Price')
plt.title('Hotel Price vs Rating')

# **Modelling**

# Train Test Split

In [ ]:
# Drop the 'Price' column to prepare data for splitting
X = master_df.drop(columns=['Price'])
# Get the target variable
y = master_df['Price']

In [ ]:
# Split data into training, validation and test sets, ensuring the class distribution is maintained
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Hyperparameter Tuning using Optuna

In [ ]:
RANDOM_SEED = 48

# 10-fold CV
kfolds = KFold(n_splits=10, shuffle=True, random_state=RANDOM_SEED)

In [ ]:
def tune(objective):
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100)

    params = study.best_params
    best_score = study.best_value
    print(f"Best score: {best_score} \nOptimized parameters: {params}")
    return params

# Ridge Regression

In [ ]:
def ridge_objective(trial):

    _alpha = trial.suggest_float("alpha", 0.1, 20)
    ridge = Ridge(alpha=_alpha, random_state=RANDOM_SEED)
    score = cross_val_score(ridge,X_train,y_train, cv=kfolds, scoring="neg_root_mean_squared_error").mean()
    return score

ridge_params = tune(ridge_objective)

# Best score: -0.13586760243668033
# ridge_params = {'alpha': 19.997759851201025}

In [ ]:
ridge = Ridge(**ridge_params, random_state=RANDOM_SEED)
ridge.fit(X_train,y_train)

In [ ]:
# Predict on the test set
y_pred_ridge = ridge.predict(X_test)

# Calculate the mean absolute error (MAE)
mae_ridge = mean_absolute_error(y_test, y_pred_ridge)
print('MAE:', mae_ridge)

In [ ]:
g_ridge = pd.DataFrame({'actual': y_test, 'predicted': y_pred_ridge})
g_ridge = g_ridge.sort_values(by="actual").reset_index()
plt.figure(figsize=(20, 5))
plt.plot(g_ridge['predicted'], label='Predicted')
plt.plot(g_ridge['actual'], color='red', label='Actual')
plt.legend(fontsize=15)
plt.title('Ridge Model - Predicted vs Actual')
plt.ylabel('Price')
plt.show()

# Lasso Regression

In [ ]:
def lasso_objective(trial):
    _alpha = trial.suggest_float("alpha", 0.0001, 1)
    lasso = Lasso(alpha=_alpha, random_state=RANDOM_SEED)
    score = cross_val_score(lasso,X_train,y_train, cv=kfolds, scoring="neg_root_mean_squared_error").mean()
    return score

lasso_params = tune(lasso_objective)

# Best score: -0.13319435700230317
# lasso_params = {'alpha': 0.0006224224345371836}

In [ ]:
lasso = Lasso(**lasso_params, random_state=RANDOM_SEED)
lasso.fit(X_train,y_train)

In [ ]:
# Predict on the test set
y_pred_lasso = lasso.predict(X_test)

# Calculate the mean absolute error (MAE)
mae_lasso = mean_absolute_error(y_test, y_pred_lasso)
print('MAE:', mae_lasso)

In [ ]:
g_lasso = pd.DataFrame({'actual': y_test, 'predicted': y_pred_lasso})
g_lasso = g_lasso.sort_values(by="actual").reset_index()
plt.figure(figsize=(20, 5))
plt.plot(g_lasso['predicted'], label='Predicted')
plt.plot(g_lasso['actual'], color='red', label='Actual')
plt.legend(fontsize=15)
plt.title('Lasso Model - Predicted vs Actual')
plt.ylabel('Price')
plt.show()

# Gradient Boosting Regressor

In [ ]:
def gbr_objective(trial):
    _n_estimators = trial.suggest_int("n_estimators", 50, 2000)
    _learning_rate = trial.suggest_float("learning_rate", 0.01, 1)
    _max_depth = trial.suggest_int("max_depth", 1, 20)
    _min_samp_split = trial.suggest_int("min_samples_split", 2, 20)
    _min_samples_leaf = trial.suggest_int("min_samples_leaf", 2, 20)
    _max_features = trial.suggest_int("max_features", 10, 50)

    gbr = GradientBoostingRegressor(
        n_estimators=_n_estimators,
        learning_rate=_learning_rate,
        max_depth=_max_depth,
        max_features=_max_features,
        min_samples_leaf=_min_samples_leaf,
        min_samples_split=_min_samp_split,
        random_state=RANDOM_SEED)

    score = cross_val_score(gbr, X_train,y_train, cv=kfolds, scoring="neg_root_mean_squared_error").mean()
    return score

gbr_params = tune(gbr_objective)

In [ ]:
gbr = GradientBoostingRegressor(random_state=RANDOM_SEED, **gbr_params)
gbr.fit(X_train,y_train)

In [ ]:
# Predict on the test set
y_pred_gbr = gbr.predict(X_test)

# Calculate the mean absolute error (MAE)
mae_gbr = mean_absolute_error(y_test, y_pred_gbr)
print('MAE:', mae_gbr)

In [ ]:
g_gbr = pd.DataFrame({'actual': y_test, 'predicted': y_pred_gbr})
g_gbr = g_gbr.sort_values(by="actual").reset_index()
plt.figure(figsize=(20, 5))
plt.plot(g_gbr['predicted'], label='Predicted')
plt.plot(g_gbr['actual'], color='red', label='Actual')
plt.legend(fontsize=15)
plt.title('Gradient Boosting Model - Predicted vs Actual')
plt.ylabel('Price')
plt.show()

# XGBRegressor

In [ ]:
def xgb_objective(trial):
    _n_estimators = trial.suggest_int("n_estimators", 50, 2000)
    _max_depth = trial.suggest_int("max_depth", 1, 20)
    _learning_rate = trial.suggest_float("learning_rate", 0.01, 1)
    _gamma = trial.suggest_float("gamma", 0.01, 1)
    _min_child_weight = trial.suggest_float("min_child_weight", 0.1, 10)
    _subsample = trial.suggest_float('subsample', 0.01, 1)
    _reg_alpha = trial.suggest_float('reg_alpha', 0.01, 10)
    _reg_lambda = trial.suggest_float('reg_lambda', 0.01, 10)


    xgbr = xgb.XGBRegressor(
        n_estimators=_n_estimators,
        max_depth=_max_depth,
        learning_rate=_learning_rate,
        gamma=_gamma,
        min_child_weight=_min_child_weight,
        subsample=_subsample,
        reg_alpha=_reg_alpha,
        reg_lambda=_reg_lambda,
        random_state=RANDOM_SEED)

    score = cross_val_score(xgbr, X_train,y_train, cv=kfolds, scoring="neg_root_mean_squared_error").mean()
    return score

xgb_params = tune(xgb_objective)

In [ ]:
xgbr = xgb.XGBRegressor(random_state=RANDOM_SEED, **xgb_params)
xgbr.fit(X_train,y_train)

In [ ]:
# Predict on the test set
y_pred_xgbr = xgbr.predict(X_test)

# Calculate the mean absolute error (MAE)
mae_xgbr = mean_absolute_error(y_test, y_pred_xgbr)
print('MAE:', mae_xgbr)

In [ ]:
g_xgbr = pd.DataFrame({'actual' : y_test,'predicted': y_pred_xgbr})
g_xgbr = g_xgbr.sort_values(by="actual").reset_index()
plt.figure(figsize = (20,5))
plt.plot(g_xgbr['predicted'])
plt.plot(g_xgbr['actual'], color = 'red')
plt.ylim(0,1000)
plt.legend(['Predicted','Actual'],fontsize = 15)
plt.title('XGBoost Model')
plt.ylabel('Price')
plt.show()

# LGBMRegressor

In [ ]:
import lightgbm as lgb

def lgb_objective(trial):
    _num_leaves = trial.suggest_int("num_leaves", 50, 100)
    _max_depth = trial.suggest_int("max_depth", 1, 20)
    _learning_rate = trial.suggest_float("learning_rate", 0.01, 1)
    _n_estimators = trial.suggest_int("n_estimators", 50, 2000)
    _min_child_weight = trial.suggest_float("min_child_weight", 0.1, 10)
    _reg_alpha = trial.suggest_float('reg_alpha', 0.01, 10)
    _reg_lambda = trial.suggest_float('reg_lambda', 0.01, 10)
    _subsample = trial.suggest_float('subsample', 0.01, 1)



    lgbr = lgb.LGBMRegressor(objective='regression',
                             num_leaves=_num_leaves,
                             max_depth=_max_depth,
                             learning_rate=_learning_rate,
                             n_estimators=_n_estimators,
                             min_child_weight=_min_child_weight,
                             subsample=_subsample,
                             reg_alpha=_reg_alpha,
                             reg_lambda=_reg_lambda,
                             random_state=RANDOM_SEED)


    score = cross_val_score(lgbr, X_train,y_train, cv=kfolds, scoring="neg_root_mean_squared_error").mean()
    return score

lgb_params = tune(lgb_objective)

In [ ]:
lgbr = lgb.LGBMRegressor(objective='regression', random_state=RANDOM_SEED, **lgb_params)
lgbr.fit(X_train,y_train)

In [ ]:
# Predict on the test set
y_pred_lgbr = lgbr.predict(X_test)

# Calculate the mean absolute error (MAE)
mae_lgbr = mean_absolute_error(y_test, y_pred_lgbr)
print('MAE:', mae_lgbr)

In [ ]:
g_lgbr = pd.DataFrame({'actual' : y_test,'predicted': y_pred_lgbr})
g_lgbr = g_lgbr.sort_values(by="actual").reset_index()
plt.figure(figsize = (20,5))
plt.plot(g_lgbr['predicted'])
plt.plot(g_lgbr['actual'], color = 'red')
plt.ylim(0,1000)
plt.legend(['Predicted','Actual'],fontsize = 15)
plt.title('LGBM Regressor Model')
plt.ylabel('Price')
plt.show()

# LGBM Regressor

In [ ]:
import optuna
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [ ]:
def objective(trial,data=X,target=y):
    param = {
        'metric': 'mae',
        'random_state': 48,
        'n_estimators': 20000,
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.006,0.008,0.01,0.014,0.017,0.02]),
        'max_depth': trial.suggest_categorical('max_depth', [10,20,100]),
        'num_leaves' : trial.suggest_int('num_leaves', 1, 1000),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 300),
        'cat_smooth' : trial.suggest_int('min_data_per_groups', 1, 100)
    }
    model1 = LGBMRegressor(**param)

    model1.fit(X_train,y_train,eval_set=[(X_test,y_test)],early_stopping_rounds=100,verbose=False)

    preds = model1.predict(X_test)

    mae = mean_absolute_error(y_test, preds)

    return mae

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

In [ ]:
# Define the best parameters obtained from Optuna
best_params = study.best_params
best_params['random_state'] = 48
best_params['n_estimators'] = 20000
best_params['metric'] = 'mae'

# Initialize and fit the LGBMRegressor model with the best parameters
model1 = LGBMRegressor(**best_params)
model1.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=100, verbose=False)

# Predict on the test set
y_pred = model1.predict(X_test)

# Evaluate the model performance
mae = mean_absolute_error(y_test, y_pred)
print('MAE:', mae)

In [ ]:
g_lgb = pd.DataFrame({'actual' : y_test,'predicted': y_pred})
g_lgb = g_lgb.sort_values(by="actual").reset_index()
plt.figure(figsize = (20,5))
plt.plot(g_lgb['predicted'])
plt.plot(g_lgb['actual'], color = 'red')
plt.ylim(0,1000)
plt.legend(['Predicted','Actual'],fontsize = 15)
plt.title('LGBM Regressor Model')
plt.ylabel('Price')
plt.show()

In [ ]:
from optuna.integration import lightgbm as lgb
lgb.plot_importance(model1, max_num_features=10, figsize=(10,10))
plt.show()

# XGBoost Regressor

In [ ]:
import xgboost as xgb
from optuna.samplers import TPESampler
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, KFold
from sklearn.metrics import mean_squared_error

In [ ]:
def objective(trial):
    param_grid = {
        'tweedie_variance_power': trial.suggest_discrete_uniform('tweedie_variance_power', 1.0, 2.0, 0.1),
        'max_depth': trial.suggest_int('max_depth', 6, 10), # Extremely prone to overfitting!
        'n_estimators': trial.suggest_int('n_estimators', 400, 4000, 400), # Extremely prone to overfitting!
        'eta': trial.suggest_float('eta', 0.007, 0.013), # Most important parameter.
        'subsample': trial.suggest_discrete_uniform('subsample', 0.2, 0.9, 0.1),
        'colsample_bytree': trial.suggest_discrete_uniform('colsample_bytree', 0.2, 0.9, 0.1),
        'colsample_bylevel': trial.suggest_discrete_uniform('colsample_bylevel', 0.2, 0.9, 0.1),
        'min_child_weight': trial.suggest_loguniform('min_child_weight', 1e-4, 1e4), # I've had trouble with LB score until tuning this.
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-4, 1e4), # L2 regularization
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-4, 1e4), # L1 regularization
        'gamma': trial.suggest_loguniform('gamma', 1e-4, 1e4),
    }

    reg = xgb.XGBModel(
        # These parameters should help with trial speed.
        objective='reg:tweedie',
        tree_method='gpu_hist',
        predictor='gpu_predictor',
        n_jobs=4,
        **param_grid
    )

    reg.fit(X_train, y_train,
            eval_set=[(X_test, y_test)], eval_metric='mae',
            verbose=False)

    # Returns the best MAE for the trial.
    # Readers may want to try returning a cross validation score here.
    return mae(y_test, reg.predict(X_test))

In [ ]:
train_time = 1 * 60 * 60
study = optuna.create_study(direction='minimize', sampler=TPESampler(), study_name='XGBRegressor')
study.optimize(objective)
print('Number of finished trials: ', len(study.trials))
print('Best trial:')

In [ ]:
# Fetch the best trial parameters and set some settings for the KFold predictions.
xgb_params = study.best_trial
xgb_params['objective'] = 'reg:tweedie'
xgb_params['tree_method'] = 'gpu_hist'
xgb_params['predictor'] = 'gpu_predictor'
xgb_params['n_jobs'] = 4

# Initialize and fit the XGBoostRegressor model with the best parameters
model2 = xgb.XGBModel(**xgb_params)
model2.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=100, verbose=False)

# Predict on the test set
y_pred = model2.predict(X_test)

# Evaluate the model performance
mae = mean_absolute_error(y_test, y_pred)
print('MAE:', mae)

In [ ]:
from optuna.integration import xgboost as xgb
xgb.plot_importance(model2, max_num_features=10, figsize=(10,10))
plt.show()

# Random Forest Regressor

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation,
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

In [ ]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('MAE: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    return accuracy

base_model = RandomForestRegressor(n_estimators = 10, random_state = 42)
base_model.fit(X_train, y_train)
base_accuracy = evaluate(base_model, X_test, y_test)

In [ ]:
best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, X_test, y_test)

# **Predict Data Test_Set_Features**

In [ ]:
test_df = pd.read_csv("test_set_features.csv")

# Mengganti Data Missing Values

In [ ]:
# Nan: Diasumsikan lupa / tidak tahu berarti ada kemungkinan memilihnya tidak berdasarkan hasil rekomendasi
# sedemikian sehingga diasumsikan '0'
test_df.doctor_recc_h1n1.fillna(0, inplace=True)
test_df.doctor_recc_seasonal.fillna(0, inplace=True)
test_df.chronic_med_condition.fillna(0,inplace=True)

# Nan: Tidak tahu harus beropini apa
test_df.opinion_h1n1_risk.fillna(3, inplace=True)
test_df.opinion_h1n1_sick_from_vacc.fillna(3, inplace=True)
test_df.opinion_seas_risk.fillna(3, inplace=True)
test_df.opinion_h1n1_vacc_effective.fillna(3,inplace=True)
test_df.opinion_seas_sick_from_vacc.fillna(3,inplace=True)
test_df.opinion_seas_vacc_effective.fillna(3,inplace=True)

# Nan: Tidak menjawab karena beranggapan variabel ini bersifat privasi
test_df.income_poverty.fillna('privacy', inplace=True)
test_df.marital_status.fillna('privacy', inplace=True)
test_df.employment_industry.fillna('privacy',inplace=True)
test_df.employment_occupation.fillna('privacy', inplace=True)
test_df.employment_status.fillna('privacy', inplace=True)

In [ ]:
# Missing values pada features yang tidak diasumsikan, diganti menjadi modus data
test_df = test_df.fillna(test_df.mode().iloc[0])
test_df.head()

In [ ]:
test_df.isna().sum()

Berdasarkan heatmap diatas data sudah bersih dari missing values karena missing values sudah berhasil diubah berdasarkan asumsi dan modus data

# Labeling Input Data

In [ ]:
# Change type float to int
test_df = test_df.astype({"h1n1_concern":"int","h1n1_knowledge":"int","behavioral_antiviral_meds":"int","behavioral_avoidance":"int","behavioral_face_mask":"int",
                            "behavioral_wash_hands":"int","behavioral_large_gatherings":"int","behavioral_outside_home":"int","behavioral_touch_face":"int",
                            "doctor_recc_h1n1":"int","doctor_recc_seasonal":"int","chronic_med_condition":"int","child_under_6_months":"int","opinion_h1n1_vacc_effective":"int",
                            "opinion_h1n1_risk":"int","opinion_h1n1_sick_from_vacc":"int","opinion_seas_vacc_effective":"int","opinion_seas_risk":"int",
                            "opinion_seas_sick_from_vacc":"int","household_adults":"int","household_children":"int"
                            })

In [ ]:
# Label Encoder
from sklearn import preprocessing

# Loop over each column in the DataFrame where dtype is 'object'
for col in test_df.select_dtypes(include = ['object']).columns:
  # Initialize a LabelEncoder object
  label_encoder = preprocessing.LabelEncoder()
  # Fit the encoder to the unique values in the column
  label_encoder.fit(test_df[col].unique())
  # Transform the column using the encoder
  test_df[col] = label_encoder.transform(test_df[col])
  # Print the column name and the unique encoded values
  print(f"{col}: {test_df[col].unique()}")

In [ ]:
new_df = test_df[['h1n1_concern', 'h1n1_knowledge',
       'behavioral_antiviral_meds', 'behavioral_avoidance',
       'behavioral_face_mask', 'behavioral_wash_hands',
       'behavioral_large_gatherings', 'behavioral_outside_home',
       'behavioral_touch_face', 'doctor_recc_h1n1', 'doctor_recc_seasonal',
       'chronic_med_condition', 'child_under_6_months', 'health_worker',
       'health_insurance', 'opinion_h1n1_vacc_effective', 'opinion_h1n1_risk',
       'opinion_h1n1_sick_from_vacc', 'opinion_seas_vacc_effective',
       'opinion_seas_risk', 'opinion_seas_sick_from_vacc', 'age_group',
       'education', 'race', 'sex', 'income_poverty', 'marital_status',
       'rent_or_own', 'employment_status', 'hhs_geo_region', 'census_msa',
       'household_adults', 'household_children', 'employment_industry',
       'employment_occupation']]

In [ ]:
# Make predictions on the testing data
y_new_pred = rfc.predict(new_df)

# Create a DataFrame with the predicted labels
h1n1_vaccine_pred = pd.DataFrame({'h1n1_vaccine': y_new_pred})
h1n1_vaccine_pred

In [ ]:
# Make predictions on the testing data
y2_pred = rfc2.predict(new_df)

# Create a DataFrame with the predicted labels
seasonal_vaccine_pred = pd.DataFrame({'seasonal_vaccine': y2_pred})
seasonal_vaccine_pred

In [ ]:
train_df.respondent_id

In [ ]:
submission_format = pd.DataFrame([])

submission_format['respondent_id'] = test_df.respondent_id
submission_format['h1n1_vaccine'] = h1n1_vaccine_pred
submission_format['seasonal_vaccine'] = seasonal_vaccine_pred
submission_format = submission_format.set_index('respondent_id')


In [ ]:
submission_format.to_csv('submission_format.csv')

# **Daftar Pustaka**

1.   https://en.wikipedia.org/wiki/Oversampling_and_undersampling_in_data_analysis
2.   https://towardsdatascience.com/understanding-random-forest-58381e0602d2
3.   https://lazypredict.readthedocs.io/en/latest/usage.html#classification
4.   https://towardsdatascience.com/understanding-random-forest-58381e0602d2
